In [ ]:
import advent

maze = advent.get_char_grid(15).astype(int)
#maze = advent.get_char_grid('15small').astype(int)
#maze = advent.get_char_grid('15maze').astype(int) + 5
print(maze.shape)

In [ ]:
import numpy as np
def duplicate_maze(maze, x_times=5, y_times=5):
    out = np.zeros((maze.shape[0] * x_times, maze.shape[1] * y_times))
    for i in range(x_times):
        for j in range(y_times):
            out[(i*maze.shape[0]):((i+1)*maze.shape[0]),
                (j*maze.shape[1]):((j+1)*maze.shape[1])] = \
                    maze + i + j
    out = np.mod(out-1, 9) + 1
    return out

if True: # part 2
    maze = duplicate_maze(maze)

In [ ]:
def cost(node, maze): # Safe-accessor
    return maze[node[0], node[1]] if walkable(node, maze) else 999999

def walkable(node, maze):
    return (0 <= node[0] < maze.shape[0]) and (0 <= node[1] < maze.shape[1])

def adjacent(node, maze):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    return [((x+a, y+b), cost((x+a, y+b), maze)) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0)]

grid_size = (1, 1)

In [ ]:
# IMPORTANT! I tried this, didn't work somehow
# If I am bored I might compare the paths and see why this failed
# Essentially, I now create all the costs first instead of this approach,
# where I calculate the costs on the fly

mod9 = lambda n: (n-1) % 9 + 1

def cost(node, maze): # Safe-accessor
    return mod9(maze[node[0] % maze.shape[0], node[1] % maze.shape[1]] + node[0] // 10 + node[1] // 10) # if walkable(node, maze) else 999_999_999

def walkable(node, maze):
    return (0 <= node[0] < 5*maze.shape[0]) and (0 <= node[1] < 5*maze.shape[1])

def adjacent(node, maze):
    # Returns list like [(adj_i, g_i)] where g_i is cost of going from node to adj_i
    x, y = node
    nodes = [((x+a, y+b), cost((x+a, y+b), maze)) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0)]
    return nodes

grid_size = (5, 5)

In [ ]:
# Copied from 2016, 13, and performance massively improved
# by using a dictionary for f instead of a list
# Todo potentially move this to a library if possible
from tqdm.notebook import tqdm

start = (0, 0)

grid_size = grid_size  # (1, 1) for part 1. (5, 5) for part 2
target = (grid_size[0]*maze.shape[0] - 1, grid_size[1]*maze.shape[1] - 1)

# Use manhattan since cost is always >= 1
# Kinda a gross underestimation so it will end up visiting almost (>99%) of the nodes anyway
h = lambda node: abs(node[0] - target[0]) + abs(node[1] - target[1])

closed = set([])
f = {start: (start, (0, h(start)))} # n: (n, (g, h))
open_parents, closed_parents = {start: None}, {}

progress=0  # out o

with tqdm(total=1) as pbar:

    while len(f) > 0:
        current_node, current_f = min(f.values(), key=lambda n: sum(n[1]))
        del f[current_node]
        closed.add(current_node)
        closed_parents[current_node] = open_parents[current_node]

        if current_node == target:
            print(f"Final path length: {sum(current_f)}")
            break

        # Here are our two day-specific functions, edit their signature if neccecary
        for adj, adj_g in adjacent(current_node, maze):
            if not walkable(adj, maze) or adj in closed:
                continue
            new_f = (current_f[0] + adj_g, h(adj))
            if (adj not in f) or (sum(new_f) < sum(f[adj][1])):
                f[adj] = (adj, new_f)
                open_parents[adj] = current_node
        
        # Update progress bar
        progress_tmp = len(closed) / ((target[0]+1) * (target[1]+1))
        if progress_tmp > progress:
            pbar.update(progress_tmp - progress)
            progress = progress_tmp

In [ ]:
# Calculate the path through the maze. Not needed for the solution, only for debugging

nodes = [target]
d = []
c = target
while True:
    c = closed_parents[c]
    if c is None: break
    d.append((c[0] - nodes[-1][0], c[1] - nodes[-1][1]))
    nodes.append(c)

import numpy as np
viz = np.array([['.'] * 50] * 50)
viz[:, :] = '.'
for n in nodes:
    viz[n] = 'x'


In [ ]:
# Show the maze
[''.join(v) for v in viz]

In [ ]:
# Let's check our answer with some half-assed dynamic programming
# again assuming you can only go down and right, then we can calculate the cost of a tile as cost(tile) + min(cost(left), cost(up))
# Therefore, this answer will be wrong, since that assumption is in general incorrect
import numpy as np

costs = np.zeros((maze.shape[0] * 5, maze.shape[1] * 5))
cost((0, 45), maze)

def get_cost_safe(node, costs):
    y, x = node
    if walkable((y, x), costs): return costs[y, x]
    return 999_999_999

for x in range(maze.shape[1] * 5):
    for y in range(maze.shape[0] * 5):
        if x == 0 and y == 0:
            # To avoid the otherwise high cost for this node
            costs[0, 0] = 0
            continue
        costs[x, y] = cost((x, y), maze) + min(get_cost_safe((x-1, y), costs), get_cost_safe((x, y-1), costs))

print(costs[-1, -1])